In [1]:
import pandas as pd 
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler,normalize

In [2]:
set_1 = pd.read_csv('mean_ratings_set1.csv')
set_2 = pd.read_csv('mean_ratings_set2.csv')

In [3]:
#Removendo linhas que possuam atributo target == Happy e atributo target == Sad, pois são multiclasse

set_1 = set_1[set_1['TARGET'] != 'HAPPY']
set_2 = set_2[set_2['TARGET'] != 'HAPPY']

set_1 = set_1[set_1['TARGET'] != 'SAD']
set_2 = set_2[set_2['TARGET'] != 'SAD']

set_2 = set_2.drop(['soundtrack', 'link','Number'], axis=1) #Possuem colunas do tipo string --> não  são necessária para o PCA
set_1 = set_1.drop(['number'],axis=1)



In [4]:
set_1.head()

,valence,energy,tension,anger,fear,happy,sad,tender,TARGET
60,5.80,2.80,2.4,1.0,1.0,5.83,2.00,7.83,TENDER
61,6.83,4.33,2.5,1.0,1.0,2.20,2.80,7.60,TENDER
62,5.83,2.00,2.0,1.0,1.0,1.60,4.40,7.40,TENDER
63,6.60,2.40,1.2,1.0,1.0,4.17,1.50,7.17,TENDER
64,6.40,3.20,1.8,1.0,1.0,4.50,2.17,7.17,TENDER


In [5]:
set_2.head()

,Valence,Energy,Tension,Anger,Fear,Happy,Sad,Tender,Beauty,Liking,TARGET
0,2.56,7.73,8.38,6.39,4.40,1.21,1.75,1.01,4.71,4.97,ANGER_HIGH
1,2.50,8.21,8.56,5.76,5.93,1.09,1.60,1.10,4.57,4.79,ANGER_HIGH
2,2.92,8.21,8.17,6.31,4.06,1.31,1.55,1.01,4.45,5.06,ANGER_HIGH
3,2.88,6.75,7.98,5.42,4.51,1.19,2.10,1.06,4.00,4.35,ANGER_HIGH
4,3.08,6.83,7.50,6.13,4.24,1.07,2.96,1.07,5.08,5.35,ANGER_HIGH


In [6]:

#Pegar os valores únicos de target --> Usando estrutura de dados SET python


targets_values_set1 = set((set_1['TARGET'].values))
targets_values_set2 = set((set_2['TARGET'].values))

targets_values = targets_values_set1 | targets_values_set2

targets_values


{'ANGER',
 'ANGER_HIGH',
 'ANGER_MODERATE',
 'ENERGY NEG HIGH',
 'ENERGY NEG MODERATE',
 'ENERGY POS HIGH',
 'ENERGY POS MODERATE',
 'FEAR',
 'FEAR_HIGH',
 'FEAR_MODERATE',
 'HAPPY_HIGH',
 'HAPPY_MODERATE',
 'HIGH ENERGY',
 'HIGH TENSION',
 'HIGH VALENCE',
 'LOW ENERGY',
 'LOW TENSION',
 'LOW VALENCE',
 'SAD_HIGH',
 'SAD_MODERATE',
 'SURPRISE',
 'TENDER',
 'TENDER_HIGH',
 'TENDER_MODERATE',
 'TENSION NEG HIGH',
 'TENSION NEG MODERATE',
 'TENSION POS HIGH',
 'TENSION POS MODERATE',
 'VALENCE NEG HIGH',
 'VALENCE NEG MODERATE',
 'VALENCE POS HIGH',
 'VALENCE POS MODERATE'}

In [7]:
#DECLARAÇÃO DE VARIAVEIS GLOBAIS PARA USO EM OUTROS FUNCOES


#Obter sentimentos base usando o target_values declarado anteriormente
modificadores = ['LOW','HIGH','MODERATE','POS','NEG'] #modificadores que não são sentimentos base
sents_base = []
for i in targets_values:   # Retira os modificadores e coloca os sentimentos base em uma lista
    for i  in (i.replace(" ","_")).split("_"):
        if i not in modificadores:
            sents_base.append(i)
sents_base = set(sents_base) #retirando duplicatas


#classificar os targets_values em 9 cores diferentes emm formato rgb

cor = [(1,0,0),(0,0,1),(1,1,0),(0,1,0),(1,0,1),(0,1,1),(0.5,0.5,0.5),(0.5,0.5,0),(0.5,0,0.5)]


sents_cor = {} #Criar um dicionário com os sentimentos base e suas respectivas cores

count = 0
for  i in sents_base:
    sents_cor[i] = cor[count] #atribuindo cores aos sentimentos base  
    count += 1
    

In [14]:
# METODOS COMPLEMENTARES PARA TRANSFORMAÇÃO LINEAR 
    
  
def rgb_resultante (rgb_base,rgb_linear):
    #Essa daqui faz o  ajuste final do vetor já normalizado e multiplicado pelas constantes  para o formato rgb
    
    rgb_r = [0,0,0]
    
    
    
    for i in range(len(rgb_base)):
        rgb_r[i] = rgb_base[i] * rgb_linear[i] * 255
    
    return rgb_r



def Padroniza_RGB_Base(sents):    
    #Vou receber um sentimento do dataframe ex: HIGH VALENCE
    #Devo retornar o rgb base contido em sents_cor com uma pequena variação de intensidade
    
    
    sents_decomposto = []
    sets_mods = []
    for i  in (sents.replace(" ","_")).split("_"):
        if i not in modificadores:
            sents_decomposto.append(i)
        else:
            sets_mods.append(i)
            
    print(sents_decomposto)
    
    print(sets_mods)
            
    if sets_mods == []:  
        return sents_cor[sents_decomposto[0]]  #sents cor [VALENCE] = (1,0,0) por exemplo
    elif sets_mods[0] == "LOW":
        
        #return np.array(sents_cor[sents_decomposto[0]]) * 0.85 
        """
        sents_cor[sents_decomposto[0]][0] = sents_cor[sents_decomposto[0]][0] * 0.85
        sents_cor[sents_decomposto[0]][1] = sents_cor[sents_decomposto[0]][1] * 0.85
        sents_cor[sents_decomposto[0]][2] = sents_cor[sents_decomposto[0]][2] * 0.85
        """
        
        return [x * 0.85 for x in sents_cor[sents_decomposto[0]]]
        
        
        
    elif sets_mods[0] == "HIGH":
        
        #return np.array(sents_cor[sents_decomposto[0]]) * 1.15
        """
        sents_cor[sents_decomposto[0]][0] = sents_cor[sents_decomposto[0]][0] * 1.15
        sents_cor[sents_decomposto[0]][1] = sents_cor[sents_decomposto[0]][1] * 1.15
        sents_cor[sents_decomposto[0]][2] = sents_cor[sents_decomposto[0]][2] * 1.15
        """
        
        return [x * 1.15 for x in sents_cor[sents_decomposto[0]]]
        
        
    
    elif sets_mods[0] == "MODERATE":
        
        #return np.array(sents_cor[sents_decomposto[0]]) * 1.00
        """
        sents_cor[sents_decomposto[0]][0] = sents_cor[sents_decomposto[0]][0] * 1.00
        sents_cor[sents_decomposto[0]][1] = sents_cor[sents_decomposto[0]][1] * 1.00
        sents_cor[sents_decomposto[0]][2] = sents_cor[sents_decomposto[0]][2] * 1.00
        """
        
        return [x * 1.00 for x in sents_cor[sents_decomposto[0]]]
        
        

    elif sets_mods[0] == "POS":
                
        #return np.array(sents_cor[sets_mods[0]]) * 1.10
        """
        sents_cor[sents_decomposto[0]][0] = sents_cor[sents_decomposto[0]][0] * 1.10
        sents_cor[sents_decomposto[0]][1] = sents_cor[sents_decomposto[0]][1] * 1.10
        sents_cor[sents_decomposto[0]][2] = sents_cor[sents_decomposto[0]][2] * 1.10
        """
        
        return [x * 1.10 for x in sents_cor[sents_decomposto[0]]]
        
      
    
    elif  sets_mods[0] == "NEG":
        
        #fornula para inverter a cor rgb : R + 256 * G + 256 * 256 * B
        
        """
        sents_cor[sents_decomposto[0]][0] = 1 - sents_cor[sents_decomposto[0]][0]
        sents_cor[sents_decomposto[0]][1] = 1 - sents_cor[sents_decomposto[0]][1]
        sents_cor[sents_decomposto[0]][2] = 1 - sents_cor[sents_decomposto[0]][2]
        """
        
        return  [1 - x for x in sents_cor[sents_decomposto[0]]]
        
     


In [9]:
def Transformacao_Linear(df):
    #Eu recebo um dataframe com as métricas e os targets, e aplico a transformação linear junto com outras funções pra adaptar os dados para o PCA
    
    
    
    
    #Separando os dados em x e y    
    y = df['TARGET']  # em y ficam os targets
    x = df.drop(columns= ['TARGET'], axis=1) # em x ficam as métricas

    

    #Normalizar valores em uma mesma escala
    Normalizado = StandardScaler()
    Normalizado.fit(x)
    escalonado = Normalizado.transform(x)

    #Apos normalização e escalonamento das métricas aplicar o pca
    num_components = 3
    pca = PCA(n_components = num_components)
    pca = pca.fit(escalonado)
    
    
    #transformando os dados em vetor rgb
    x = pca.transform(escalonado)
    
    #Normalizando os dados
    x = normalize(x)

    count = 0
    rgb = []
    for i in x:
        
        rgb.append(rgb_resultante(Padroniza_RGB_Base(y.iloc[count]) ,i))   # Problema cabeludo aqui
        count += 1
    return rgb
    
    
    
    

In [12]:
# T(set_1) = s1  e T(set_2) = s2


s1  = Transformacao_Linear(set_1)


['TENDER']
[]
['TENDER']
[]
['TENDER']
[]
['TENDER']
[]
['TENDER']
[]
['TENDER']
[]
['TENDER']
[]
['TENDER']
[]
['TENDER']
[]
['TENDER']
[]
['TENDER']
[]
['TENDER']
[]
['TENDER']
[]
['TENDER']
[]
['TENDER']
[]
['TENDER']
[]
['TENDER']
[]
['TENDER']
[]
['TENDER']
[]
['TENDER']
[]
['TENDER']
[]
['TENDER']
[]
['TENDER']
[]
['TENDER']
[]
['TENDER']
[]
['TENDER']
[]
['TENDER']
[]
['TENDER']
[]
['TENDER']
[]
['TENDER']
[]
['FEAR']
[]
['FEAR']
[]
['FEAR']
[]
['FEAR']
[]
['FEAR']
[]
['FEAR']
[]
['FEAR']
[]
['FEAR']
[]
['FEAR']
[]
['FEAR']
[]
['FEAR']
[]
['FEAR']
[]
['FEAR']
[]
['FEAR']
[]
['FEAR']
[]
['FEAR']
[]
['FEAR']
[]
['FEAR']
[]
['FEAR']
[]
['FEAR']
[]
['FEAR']
[]
['FEAR']
[]
['FEAR']
[]
['FEAR']
[]
['FEAR']
[]
['FEAR']
[]
['FEAR']
[]
['FEAR']
[]
['FEAR']
[]
['FEAR']
[]
['ANGER']
[]
['ANGER']
[]
['ANGER']
[]
['ANGER']
[]
['ANGER']
[]
['ANGER']
[]
['ANGER']
[]
['ANGER']
[]
['ANGER']
[]
['ANGER']
[]
['ANGER']
[]
['ANGER']
[]
['ANGER']
[]
['ANGER']
[]
['ANGER']
[]
['ANGER']
[]
['ANGER']
[]

In [15]:
s2 = Transformacao_Linear(set_2)

['ANGER']
['HIGH']
['ANGER']
['HIGH']
['ANGER']
['HIGH']
['ANGER']
['HIGH']
['ANGER']
['HIGH']
['ANGER']
['MODERATE']
['ANGER']
['MODERATE']
['ANGER']
['MODERATE']
['ANGER']
['MODERATE']
['ANGER']
['MODERATE']
['FEAR']
['HIGH']
['FEAR']
['HIGH']
['FEAR']
['HIGH']
['FEAR']
['HIGH']
['FEAR']
['HIGH']
['FEAR']
['MODERATE']
['FEAR']
['MODERATE']
['FEAR']
['MODERATE']
['FEAR']
['MODERATE']
['FEAR']
['MODERATE']
['HAPPY']
['HIGH']
['HAPPY']
['HIGH']
['HAPPY']
['HIGH']
['HAPPY']
['HIGH']
['HAPPY']
['HIGH']
['HAPPY']
['MODERATE']
['HAPPY']
['MODERATE']
['HAPPY']
['MODERATE']
['HAPPY']
['MODERATE']
['HAPPY']
['MODERATE']
['SAD']
['HIGH']
['SAD']
['HIGH']
['SAD']
['HIGH']
['SAD']
['HIGH']
['SAD']
['HIGH']
['SAD']
['MODERATE']
['SAD']
['MODERATE']
['SAD']
['MODERATE']
['SAD']
['MODERATE']
['SAD']
['MODERATE']
['TENDER']
['HIGH']
['TENDER']
['HIGH']
['TENDER']
['HIGH']
['TENDER']
['HIGH']
['TENDER']
['HIGH']
['TENDER']
['MODERATE']
['TENDER']
['MODERATE']
['TENDER']
['MODERATE']
['TENDER']
['MODER

In [13]:
print(s1)

[[112.58223058640321, 45.08533853543659, 39.354842209504724], [119.07858130198058, 16.16704030948727, 42.60479178143014], [114.25391999696916, -42.41971483361765, 37.454766836288385], [114.7945314925265, 29.2805932418818, 47.12867915414995], [116.35372553731764, 36.05545015842453, 37.657204721804824], [115.33913184078992, 28.138320237726862, 46.490532373936254], [113.4304492300939, 48.216681753850196, 32.63333859884734], [117.06453304513366, 25.989815083298343, 43.32060266040593], [114.1451664853326, 38.0552395862669, 42.17736013630451], [119.14205476049251, 6.069158354610119, 44.99540092403292], [120.9986522959288, -8.497121963814442, 39.28581246329242], [114.2596096280517, 40.26479081731056, 39.74592089867158], [121.3402443081497, 31.90716038450169, 22.68762278063147], [116.07752241345489, 44.69552536297789, 28.010155352680233], [118.71840909951075, 34.943247886986654, 30.678311035482473], [121.01885963208639, -38.178446621024264, 12.373028204727062], [104.05054130157465, 68.61706785

In [16]:
print(s2)

[[135.12111874815793, -4.160259157823607, -0.0], [140.88555499755103, -2.7486810213054036, -0.0], [134.18842156347142, 8.673658014140699, -0.0], [146.32019134891283, -8.8682548728452, -0.0], [120.30479226897528, -30.289271117112165, -0.0], [113.87723109187355, 57.34305701221737, 0.0], [102.68500169613338, 3.6374890707280914, -0.0], [47.75436491034527, -73.03106486398379, -0.0], [106.18300631585346, 35.23937939951475, -0.0], [121.31462495843324, -25.01817307090156, -0.0], [0.0, -27.2119346211314, 68.24987731557124], [0.0, 9.400851329600249, -0.7753473970144394], [0.0, 5.2732677409366016, -27.881729663448002], [0.0, 3.8281589301461447, 91.98328643969388], [0.0, -35.16646353299477, 99.16850768300831], [0.0, 35.22706212333234, 29.734923263353622], [0.0, -60.711771180928004, 100.96823146686899], [0.0, 12.33555811781446, -97.47249017383832], [0.0, -164.36490673512995, 133.4597318915834], [0.0, 79.83221603399231, -7.1574644389179225], [-0.0, 258.8462217375889, -0.0], [-0.0, 288.67071464109256